In [1]:
!pip install datasets
!pip install transformers
!pip install evaluate
!pip install rouge_score bert_score sacrebleu


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
os.environ['CURL_CA_BUNDLE'] = ''

import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [3]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn import functional as F
from transformers import BartTokenizer, BartForConditionalGeneration, AdamW, get_polynomial_decay_schedule_with_warmup
from datasets import load_dataset
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import random
from tqdm import tqdm
import sys
import math



c:\Users\aramosvela\Documents\Research_Project\research_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
dataset_name = 'daily_dialog'

In [5]:
if dataset_name == 'daily_dialog':
    print('Loading ', dataset_name)
    dataset = load_dataset('daily_dialog')
    train_dialogues = dataset['train']['dialog']
    valid_dialogues = dataset['validation']['dialog']
    test_dialogues = dataset['test']['dialog']
elif dataset_name == 'empathetic_dialogues':
    print('Loading ', dataset_name)
    dataset = load_dataset('empathetic_dialogues')
    train_dialogues = dataset['train']
    valid_dialogues = dataset['validation']
    test_dialogues = dataset['test']
else:
    print('No dataset selected')

Loading  daily_dialog


Found cached dataset daily_dialog (C:/Users/aramosvela/.cache/huggingface/datasets/daily_dialog/default/1.0.0/1d0a58c7f2a4dab5ed9d01dbde8e55e0058e589ab81fce5c2df929ea810eabcd)
100%|██████████| 3/3 [00:00<00:00, 132.02it/s]


In [7]:
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")

sp1_token = '<sp1>'
sp2_token = '<sp2>'
# bos_token = '<bos>'
# eos_token = '<eos>'
max_len = 1024
seed = 0
gpu = 0

#Tokeniser
special_tokens = {#'bos_token': bos_token,
                'additional_special_tokens': [sp1_token, sp2_token]}

eos_token = tokenizer.eos_token
num_new_tokens = tokenizer.add_special_tokens(special_tokens)

vocab = tokenizer.get_vocab()
vocab_size = len(vocab)
# bos_id = vocab[bos_token]
# eos_id = vocab[eos_token]
sp1_id = vocab[sp1_token]
sp2_id = vocab[sp2_token]

lr = 2e-5
batch_size = 8
num_workers = 0
num_epochs = 8
warmup_ratio = 0.1
last_epoch = 0
end_command = 'Quit!'
top_p = 0.8


In [36]:
vocab['<pad>']

1

In [8]:
!mkdir 'saved_models'
ckpt_dir = 'saved_models'

A subdirectory or file 'saved_models' already exists.


In [56]:
def preprocess_dialog(dialog, window_size=5):
    instances = []
    
    # response = dialog["dialog"][-1]  # Last utterance as the response

    for i in range(0, len(dialog) - window_size - 1, 2):
        
        window = dialog[i:i+window_size]
        window_context = []
        for j, utterance in enumerate(window):
            speaker = sp1_token if j % 2 == 0 else sp2_token
            window_context.append(speaker + " " + utterance)

        # Add special tokens for bos, eos
        window_context.insert(0, '<s>')
        # window_context.append("</s>")

        window_context = ' '.join(window_context)
        window_context = window_context + sp2_token
        response =  dialog[i+window_size] + '</s>'

        # print('window_context: ', type(window_context), window_context)
        # print('response: ', type(response), response)
        


        # print()
        # print('window_context: ', window_context)
        # print('response: ', response)


        # Tokenize the context and response
        input_ids = tokenizer.encode_plus(window_context, add_special_tokens=False, padding='max_length', max_length=512, truncation=True , return_tensors="pt")

        decoder_input_ids = tokenizer.encode_plus(response, add_special_tokens=False, padding='max_length', max_length=512, truncation=True, return_tensors="pt")

        labels = decoder_input_ids['input_ids']

        labels[labels[:, :] == vocab['<pad>']] = -100

        instance = {
            "input_ids": input_ids["input_ids"].squeeze(0),
            "attention_mask": input_ids["attention_mask"].squeeze(0),
            "labels": labels.squeeze(0)
            # "decoder_input_ids": decoder_input_ids["input_ids"].squeeze(0),
            # "decoder_attention_mask": decoder_input_ids["attention_mask"].squeeze(0)
            
        }

        # print('input_ids: ', instance['input_ids'].shape)
        # print('attention_mask: ', instance['attention_mask'].shape)
        # print('decoder_input_ids: ', instance['decoder_input_ids'].shape)
        # print('decoder_attention_mask: ', instance['decoder_attention_mask'].shape)
        instances.append(instance)
        



    return instances

In [57]:
train_dialogues[0]

['Say , Jim , how about going for a few beers after dinner ? ',
 ' You know that is tempting but is really not good for our fitness . ',
 ' What do you mean ? It will help us to relax . ',
 " Do you really think so ? I don't . It will just make us fat and act silly . Remember last time ? ",
 " I guess you are right.But what shall we do ? I don't feel like sitting at home . ",
 ' I suggest a walk over to the gym where we can play singsong and meet some of our friends . ',
 " That's a good idea . I hear Mary and Sally often go there to play pingpong.Perhaps we can make a foursome with them . ",
 ' Sounds great to me ! If they are willing , we could ask them to go dancing with us.That is excellent exercise and fun , too . ',
 " Good.Let ' s go now . ",
 ' All right . ']

In [58]:
train_instances = []
val_instances = []

#dummy
test_instances = []

for dialog in tqdm(train_dialogues):
    train_instances.extend(preprocess_dialog(dialog))
    # break

for dialog in tqdm(valid_dialogues):
    val_instances.extend(preprocess_dialog(dialog))

for dialog in tqdm(test_dialogues):
    test_instances.extend(preprocess_dialog(dialog))

100%|██████████| 1000/1000 [00:01<00:00, 853.29it/s]


In [59]:
train_instances[0]['input_ids']

tensor([    0, 50265, 34673,  2156,  2488,  2156,   141,    59,   164,    13,
           10,   367, 16328,    71,  3630, 17487, 50266,  1185,   216,    14,
           16, 25057,    53,    16,   269,    45,   205,    13,    84,  5704,
          479, 50265,  2264,   109,    47,  1266, 17487,    85,    40,   244,
          201,     7, 12327,   479, 50266,  8275,    47,   269,   206,    98,
        17487,    38,   218,    75,   479,    85,    40,    95,   146,   201,
         5886,     8,  1760, 15470,   479,  9427,    94,    86, 17487, 50265,
          100,  4443,    47,    32,   235,     4,  1708,    99,  5658,    52,
          109, 17487,    38,   218,    75,   619,   101,  2828,    23,   184,
          479, 50266,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1, 

In [60]:
train_instances[0]['labels']

tensor([   38,  3608,    10,  1656,    81,     7,     5,  6545,   147,    52,
           64,   310, 22707,  1657,     8,   972,   103,     9,    84,   964,
          479,  1437,     2,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100, 

In [15]:
# tokenizer.decode(train_instances[0]['decoder_input_ids'], skip_special_tokens=True , clean_up_tokenization_spaces=False)

' I suggest a walk over to the gym where we can play singsong and meet some of our friends . '

In [61]:
class DialogueDataset(Dataset):
    def __init__(self, instances):
        self.instances = instances

    def __len__(self):
        return len(self.instances)

    def __getitem__(self, idx):
        return self.instances[idx]



In [62]:
#Create data loaders
train_dataset = DialogueDataset(train_instances)
val_dataset = DialogueDataset(val_instances)
test_dataset =  DialogueDataset(test_instances)

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False)


In [63]:
print(train_dataset.__getitem__(0)['input_ids'].shape)
print(train_dataset.__getitem__(0)['attention_mask'].shape)

# print(train_dataset.__getitem__(0)['decoder_input_ids'].shape)
# print(train_dataset.__getitem__(0)['decoder_attention_mask'].shape)


torch.Size([512])
torch.Size([512])


In [64]:
print(train_dataset.__getitem__(1)['input_ids'].shape)
print(train_dataset.__getitem__(1)['attention_mask'].shape)

# print(train_dataset.__getitem__(1)['decoder_input_ids'].shape)
# print(train_dataset.__getitem__(1)['decoder_attention_mask'].shape)

torch.Size([512])
torch.Size([512])


In [65]:
def fix_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    random.seed(seed)

In [66]:
if torch.cuda.is_available():
    device = torch.device(f"cuda:{gpu}")
    print('Using GPU')
else:
    device = torch.device("cpu")
    print('Using CPU')

Using CPU


In [67]:
#Define the BART model and optimizer

fix_seed(seed)
model = BartForConditionalGeneration.from_pretrained("facebook/bart-base").to(device)
model.resize_token_embeddings(vocab_size)

Embedding(50267, 768)

In [68]:
print("Loading the optimizer...")
optim = torch.optim.AdamW(model.parameters(), lr=lr)


Loading the optimizer...


In [69]:
# Calculate total training steps
num_batches = len(test_dataloader) #len(train_dataloader) ###CHANGE LATER
total_train_steps = num_epochs * num_batches
warmup_steps = int(warmup_ratio * total_train_steps)

sched = get_polynomial_decay_schedule_with_warmup(
    optim,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_train_steps,
    power=2
)

writer = SummaryWriter()


In [70]:
def validation():

    print("Validation processing...")
    model.eval()
            
    valid_losses = []
    valid_ppls = []
    with torch.no_grad():
        for i, batch in enumerate(tqdm(test_dataloader)):
            
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            # decoder_input_ids = batch["decoder_input_ids"].to(device)
            # decoder_attention_mask = batch["decoder_attention_mask"].to(device)
            
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                # decoder_input_ids=decoder_input_ids,
                # decoder_attention_mask=decoder_attention_mask,
                labels = labels
                # use_cache=False
            )
            
            loss = outputs.loss
            
            valid_losses.append(loss.detach())
            ppl = torch.exp(loss.detach())
            valid_ppls.append(ppl)
        
        valid_losses = [loss.item() for loss in valid_losses]
        valid_ppls = [ppl.item() if not math.isinf(ppl.item()) else 1e+8 for ppl in valid_ppls]
        valid_loss = np.mean(valid_losses)
        valid_ppl = np.mean(valid_ppls)
        
        if math.isnan(valid_ppl):
            valid_ppl = 1e+8
            
    return valid_loss, valid_ppl

In [71]:
def train():

    fix_seed(seed)  # Fix seed before training
    print("Training starts.")

    best_loss = sys.float_info.max
    last_epoch= 0

    start_epoch = last_epoch +1

    for epoch in range(start_epoch, start_epoch+num_epochs):
        model.train()

        print(f"#"*50 + f"Epoch: {epoch}" + "#"*50)
        train_losses = []
        train_ppls = []

        # total_loss = 0

        for batch in tqdm(val_dataloader):

            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            # decoder_input_ids = batch["decoder_input_ids"].to(device)
            # decoder_attention_mask = batch["decoder_attention_mask"].to(device)

            optim.zero_grad()


            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                # decoder_input_ids=decoder_input_ids,
                # decoder_attention_mask=decoder_attention_mask,
                labels = labels
                # use_cache=False
            )

            loss = outputs.loss
            loss.backward()
            optim.step()
            sched.step()

            train_losses.append(loss.detach())
            ppl = torch.exp(loss.detach())
            train_ppls.append(ppl)
        
        train_losses = [loss.item() for loss in train_losses]
        train_ppls = [ppl.item() if not math.isinf(ppl.item()) else 1e+8 for ppl in train_ppls]
        train_loss = np.mean(train_losses)
        train_ppl = np.mean(train_ppls)
        print(f"Train loss: {train_loss} || Train perplexity: {train_ppl}")

        writer.add_scalar("Loss/train", train_loss, epoch)
        writer.add_scalar("PPL/train", train_ppl, epoch)
        
        last_epoch += 1
        
        valid_loss, valid_ppl = validation()

        if valid_loss < best_loss:
            best_loss = valid_loss
            state_dict = {
                'model_state_dict': model.state_dict(),
                'optim_state_dict': optim.state_dict(),
                'sched_state_dict': sched.state_dict(),
                'loss': best_loss,
                'epoch': last_epoch
            }
            
            torch.save(state_dict, f"{ckpt_dir}/best_ckpt_epoch={epoch}_valid_loss={round(best_loss, 4)}.ckpt")
            print("*"*10 + "Current best checkpoint is saved." + "*"*10)
            print(f"{ckpt_dir}/best_ckpt_epoch={epoch}_valid_loss={round(best_loss, 4)}.ckpt")
            
        print(f"Best valid loss: {best_loss}")
        print(f"Valid loss: {valid_loss} || Valid perplexity: {valid_ppl}")
        
        writer.add_scalar("Loss/valid", valid_loss, epoch)
        writer.add_scalar("PPL/valid", valid_ppl, epoch)
        
        writer.add_scalars("Losses", {
            'train': train_loss, 
            'valid': valid_loss,
        }, epoch)
        writer.add_scalars("PPLs", {
            'train': train_ppl,
            'valid': valid_ppl,
        }, epoch)
            
    print("Training finished!")

In [72]:
num_epochs=1
train()

Training starts.
##################################################Epoch: 1##################################################


  5%|▌         | 10/184 [03:20<58:05, 20.03s/it] 


KeyboardInterrupt: 

In [73]:
def infer(window_size=5):
    model.eval()
    fix_seed(seed)

    generated_responses = []
    actual_responses = []

    with torch.no_grad():

        for i, batch in enumerate(tqdm(test_dialogues)):

            for i in range(len(batch) - window_size - 1):
                
                window = dialog[i:i+window_size]
                window_context = []
                for j, utterance in enumerate(window):
                    speaker = sp1_token if j % 2 == 0 else sp2_token
                    window_context.append(speaker + " " + utterance)

                # Add special tokens for bos, eos
                window_context.insert(0, '<s>')
                # window_context.append("</s>")

                window_context = ' '.join(window_context)
                window_context = window_context + sp2_token

                print('window context: ', window_context)

                #Get encodings
                encodings = tokenizer.encode_plus(window_context, add_special_tokens=False, padding='max_length', max_length=512, truncation=True , return_tensors="pt")
                
                input_ids = encodings['input_ids']
                attention_mask = encodings['attention_mask']

                print('input_ids: ', input_ids.shape, input_ids)
                print('attention_mask: ', attention_mask.shape, attention_mask)

                output_ids = model.generate(input_ids, max_length=512, do_sample=True, top_p=top_p).squeeze(0)

                #Generate response
                # output_ids = model(input_ids=input_ids, attention_mask=attention_mask)

                print('encoded response: ', output_ids)
                # print('encoded response: ', output_ids.squeeze(0))

                response = tokenizer.decode(output_ids, skip_special_tokens=True)

                print('generated response: ', response)

                actual_response = dialog[i+window_size]

                print('actual_response: ', actual_response)

                generated_responses.append(response)
                actual_responses.append(actual_response)

                break
            break

def nucleus_sampling(input_ids, attention_mask): #token_type_ids, input_len, next_sp_id):

    output_ids = []
    for pos in range(len(input_ids), max_len):

        # output = model(input_ids=input_ids, attention_mask=attention_mask)[0][:, pos-1] 
        # print('debug: ', output[0])
        # break
        output = model(input_ids=input_ids, attention_mask=attention_mask)[0][:, pos-1]  # (1, V)
        output = F.softmax(output, dim=-1)  # (1, V)

    
        
        sorted_probs, sorted_idxs = torch.sort(output, descending=True)
        cumsum_probs = torch.cumsum(sorted_probs, dim=-1)  # (1, V)
        idx_remove = cumsum_probs > top_p
        idx_remove[:, 1:] = idx_remove[:, :-1].clone()
        idx_remove[:, 0] = False
        sorted_probs[idx_remove] = 0.0
        sorted_probs /= torch.sum(sorted_probs, dim=-1, keepdim=True)  # (1, V)
        
        probs = torch.zeros(output.shape, device=device).scatter_(-1, sorted_idxs, sorted_probs)  # (1, V)
        idx = torch.multinomial(probs, 1)  # (1, 1)
        
        idx_item = idx.squeeze(-1).squeeze(-1).item()
        output_ids.append(idx_item)

        print('xd')
        
        if idx_item == vocab['</s>']:
            break
            
        # input_ids = torch.cat((input_ids, idx), dim=-1)

        # next_type_id = torch.LongTensor([[next_sp_id]]).to(device)
        # token_type_ids = torch.cat((token_type_ids, next_type_id), dim=-1)
        # assert input_ids.shape == token_type_ids.shape
        
    return output_ids

In [74]:
infer()

  0%|          | 0/1000 [00:00<?, ?it/s]

window context:  <s> <sp1> What a nice day !  <sp2>  yes . How about going out and enjoying the sunshine on the grass ?  <sp1>  great , let's go !  <sp2>  hey , darling , I think I might have a little heatstroke from being in the sun all day . I was so relaxed . It felt as if I were in another world .  <sp1>  exactly . You know , the sunshine and wind remind me of our honeymoon . You remember ? The island , the sound of the waves , the salty sea air and the sunshine ... <sp2>
input_ids:  torch.Size([1, 512]) tensor([[    0, 50265,  2264,    10,  2579,   183, 27785, 50266, 10932,   479,
          1336,    59,   164,    66,     8,  6218,     5, 15049,    15,     5,
          6964, 17487, 50265, 12338,  2156,   905,    18,   213, 27785, 50266,
         12229,  2156, 30213,  2156,    38,   206,    38,   429,    33,    10,
           410,  2859, 17097,    31,   145,    11,     5,  3778,    70,   183,
           479,    38,    21,    98, 11956,   479,    85,  1299,    25,   114,
            

  0%|          | 0/1000 [00:02<?, ?it/s]

encoded response:  tensor([    2,     0,  1437,  2264,    10,  2579,   183, 27785,  1437,  1437,
         1437,    38,   206,    38,   429,    33,    10,   410,  2859, 17097,
           31,   145,    11,     5,  3778,    70,   183,   479,  1437,  1437,
          653,    10,  9869,   183, 27785, 17487,  1437,  1437,     2])
generated response:   What a nice day!    I think I might have a little heatstroke from being in the sun all day.   What a lovely day!?  
actual_response:   yes , it was wonderful but it's already been a year . How time flies ! 


In [75]:
def generate_response(context):
    model.eval()
    input_ids = tokenizer.encode(context, padding='max_length', max_length=512, truncation=True , return_tensors="pt").to(device)
    output_ids = model.generate(input_ids=input_ids, max_length=512, )
    response = tokenizer.decode(output_ids.squeeze(), skip_special_tokens=True)

    return response

context = "<sp1> How are you? <sp2> I'm doing well. How about you? <sp1> I'm good too. Thanks for asking. <sp2>"

response = generate_response(context)
print("Generated Response:", response)

Generated Response: How are you? I'm doing well. How about you? Me too. I'm good too. Thanks for asking.
